In [ ]:
<img src="https://www.mines.edu/webcentral/wp-content/uploads/sites/267/2019/02/horizontallightbackground.jpg" width="100%"> 

### CSCI250 Python Computing: Building a Sensor System
<hr style="height:5px" width="100%" align="left">

# Analog/Digital Converter

# Introduction

<img src="https://cdn.sparkfun.com//assets/parts/1/7/7/0/08636-03-L.jpg" width="30%" align='right'>

**Analog to Digital Converters** (ADC) are devices that convert analog signals into digital signals.

This is useful because many natural phenomena are characterized by continuous signals, while processing them is digital (discretized).

More information about ADC chips is available on [this page](https://en.wikipedia.org/wiki/Analog-to-digital_converter).

# Theory

The key component of an ADC is a **comparator**, which outputs a HIGH/LOW voltage if the input voltage is higher/lower than a reference. 

* **VREF** (reference voltage 3.3V) has a chain of resistors creating a voltage divider, with the center node at 1.65V. 
* If the voltage on channel 0 (**CHO**) is above 1.65V, the comparator outputs HIGH voltage; otherwise it outputs LOW voltage. 

**A circuit with 1 comparator can measure 2 states.**
<img src="https://www.dropbox.com/s/bpx17rrcl1nlpzg/mcp3002_1_bit_adc.PNG?raw=1" width="30%" align="left">
<img src="https://www.dropbox.com/s/j1vdz44bixzu4d6/mcp3002_1_bit_plot.PNG?raw=1" width="70%" align="right">

For higher resolution, we can add comparators and a longer resistor chain. All resistors have the same resistance and create multiple voltage dividers (each inner node has 1/4, 2/4, and 3/4 of **VREF**). 

A comparator is located at each inner node to be compared with the input voltage and outputs HIGH when the input voltage goes above the corresponding voltage in the resistor chain.

**A circuit with 3 comparators can measure 4 states.**
<img src="https://www.dropbox.com/s/ulhj9krqchn56cw/mcp3002_2_bit_adc.PNG?raw=1" width="30%" align="left">
<img src="https://www.dropbox.com/s/6cbihzbuqoy2p90/mcp3002_2_bit_plot.PNG?raw=1" width="70%" align="right">

Multiple comparators increase resolution, but generate many outputs. 

A **decoder** compresses all outputs into a single binary number.

<img src="https://www.dropbox.com/s/7fou6f2sq8w72dy/mcp3002_2_bit_adc_decoder.PNG?raw=1" width="30%" align="left">
<img src="https://www.dropbox.com/s/gmy1tvdnwvnnchj/mcp3002_10_bit_plot.PNG?raw=1" width="70%">

# MCP3002 ADC

<img src="https://cdn.sparkfun.com//assets/parts/1/7/7/0/08636-03-L.jpg" width="30%" align='left'>

The **MCP3002 ADC** has $2^{10}=1024$ different states - a very fine resolution that is almost indistinguishable from an analog voltage.

The ADC returns integers between $0$ and $1023$.

More information is available under the documents tabs of the [SparkFun ADC](https://www.sparkfun.com/products/8636).

# Wiring (1)

Mount the ADC into your breadboard over the middle divider.

**N.B.**: Make sure the ADC notch is aligned with the diagrams. If you reverse the orientation, 3.3V and GND are reversed causing the ADC to rapidly heat up and damage it.

<img src="https://www.dropbox.com/s/xircvfi3spyc5oj/wiring_adc.PNG?raw=1" width="65%" align="left">

<img src="https://www.dropbox.com/s/6lt2y3toa5iuv5v/circuit_adc.jpg?raw=1" width="35%" align="right">

# Wiring (2)
1. **CS**: Chip select - connect to **CE0** on the Wedge.
2. **CH0**: Channel 0 Input - connect an analog sensor to this pin.
3. **CH1**: Channel 1 Input - connect an analog sensor to this pin.
4. **VSS**: Ground - connect to **GND** on the Wedge.
5. **DIN**: Serial data in - connect to **MOSI** on the Wedge.
6. **DOUT**: Serial data out - connect to **MISO** on the Wedge.
7. **CLK**: Clock Pulse - connect to **SCK** on the Wedge.
8. **VDD/VREF**: Reference voltage- connect to **3.3V** on the Wedge.

# SPI interface

The ADC communicates with the RPi over the [Serial Peripheral Interface (SPI)](https://en.wikipedia.org/wiki/Serial_Peripheral_Interface).

SPI is a digital communication protocol that allows devices to talk to each other. SPI includes 4 wires that all go HIGH/LOW at specific times to send a stream of bits. Every device on an SPI bus has a separate chip select wire, which goes LOW to indicate it's being communicated with.

<img src="http://blog.digilentinc.com/wp-content/uploads/2014/09/SPI-signals.png" width="50%">

The RPi sends 4 bits to the ADC to specify what is requested (which channel, differential measurement, etc.), after which the ADC sends 10 bits back representing the measured voltage (ranges from 0-1023).

<img src="https://www.dropbox.com/s/u628vjn2uc5h3ua/notebook.png?raw=1" width="10%" align="left">

See the [SPI notebook](h_SPI.ipynb) for more details about SPI communication with the ADC.

# ADC reader
For simplicity, we have created a module (`adcUtil.py`) with a simple function (`readADC`) that enables data collection from an analog sensor. 

`readADC` takes a channel argument and returns a voltage.

In [1]:
import adcUtil as adc

# Testing
Call the `readADC` function to check that it returns a number between $0$ and $3.3V$. 

You need one analog sensor connected to the channel used in the call. 

In [2]:
adc.readADC(channel=0)

0.0

<img src="https://www.dropbox.com/s/u628vjn2uc5h3ua/notebook.png?raw=1" width="10%" align="left">

See the notebooks for the [potentiometer](h_Potentiometer.ipynb) or the [photoresistor](h_Photoresistor.ipynb) for information about analog sensor connection.

# Troubleshooting

* **I get error messages when I run my code.**
    * There's probably an issue with your code, not any of the components. Read the error message and try to understand what it's telling you in order to fix it.
* **The ADC always returns 0.**
    * You're probably having an issue communicating with the ADC, which would most likely come from a wiring problem. Double check your wiring of the ADC to make sure the connections are correct. Also double check your code for any bugs, and that the channel you're measuring isn't just connected to 0V.
* **I get funky numbers from the ADC.**
    * The most likely cause is that the pin is left floating because it's not connected to anything; simply moving your hand near the ADC can cause the values to change. Make sure there is something connected to one of the channel pins of the ADC, and make sure your code is getting data from the same channel.
    * If this step doesn't help, try your neighbor's ADC. If that works, yours is likely broken.